# 2.2 Export segments

### 2.2.2. Make a folder for images to be exported  
First, make a folder inside your QuPath project folder named ```export```.  
This is where the exported images will be saved. Your folder structure should look like this:  

**qupath_project**
- **classifiers**
  - `classes.json`
  - **pixel_classifiers**
    - `edema_20241006.json`
    - `gaps_tutorial.json`
    - `tissue_background.json`
- **data** 
- `project.qpproj`
- `project.qpproj.backup`
- **scripts**
  - `segment_gaps.groovy`
  - `segment_tissue.groovy`
  - `training_annotation.groovy`
  - `training_annotation_gaps.groovy`
- **export** 

### 2.2.3 Export segmentation from QuPath 
Run the following script in QuPath to export the gap segmentation.  
**name_tissue**: write the name of the tissue segments here within quotation marks  
**name_gaps**: write the name of the gap segments within QuPath here, using quotation marks  

Images will be saved as tiff files. 

In [ ]:
import qupath.lib.images.servers.LabeledImageServer

name_tissue = 'tissue'
name_gaps = 'fluid'


fluid = getAnnotationObjects().findAll{it.getPathClass() == getPathClass(name_gaps)}


if (fluid) {
    
    def imageData = getCurrentImageData()
    
    // Define output path (relative to project)
    def name = GeneralTools.getNameWithoutExtension(imageData.getServer().getMetadata().getName())
    def pathOutput = buildFilePath(PROJECT_BASE_DIR, 'export', name)
    mkdirs(pathOutput)
    
    // Define output resolution
    // double requestedPixelSize = 1.0
    
    // Convert to downsample. Factor of 1 means no downsampling. 
    double downsample = 1.0 //requestedPixelSize / imageData.getServer().getPixelCalibration().getAveragedPixelSize()
    
    // Create an ImageServer where the pixels are derived from annotations
    def edemaServer = new LabeledImageServer.Builder(imageData)
        .backgroundLabel(0, ColorTools.WHITE) // Specify background label (usually 0 or 255)
        .downsample(downsample)    // Choose server resolution; this should match the resolution at which tiles are exported
        .addLabel(name_gaps, 1)
        .grayscale()
        .multichannelOutput(false) // If true, each label refers to the channel of a multichannel binary image (required for multiclass probability)
        .build()
    
    def tissueServer = new LabeledImageServer.Builder(imageData)
        .backgroundLabel(0, ColorTools.WHITE) // Specify background label (usually 0 or 255)
        .downsample(downsample)    // Choose server resolution; this should match the resolution at which tiles are exported
        .addLabel(name_tissue, 1)      // Choose output labels (the order matters!)
        .grayscale()
        .multichannelOutput(false) // If true, each label refers to the channel of a multichannel binary image (required for multiclass probability)
        .build()
    
    
    // Export things within the tissue bounding box 
    annotation = getAnnotationObjects().find{it.getPathClass() == getPathClass(name_tissue)}
    def edema_region = RegionRequest.createInstance(edemaServer.getPath(), downsample, annotation.getROI())
    def tissue_region = RegionRequest.createInstance(tissueServer.getPath(), downsample, annotation.getROI())
    def tissuePath = buildFilePath(pathOutput, name_tissue + '.tif')
    def edemaPath = buildFilePath(pathOutput, name_gaps + '.tif')

    
    writeImageRegion(edemaServer, edema_region, edemaPath)
    writeImageRegion(tissueServer, tissue_region, tissuePath)
    
 }

### Checking the output 
The export folder (see above) should now have the following structure: 
* A folder for each image that was processed. In each of which:  
* Two files: One containing the tissue mask (below as "tissue.tif"), the other gap segmentation mask (below as "fluid.tif")


- **export** 
    - `name_image_1`
      - `fluid.tif` 
      - `tissue.tif`
    - `name_image_2`
    - `name_image_n`